## Importing Libraries

In [1]:
!pip install PyPDFLoader --user

ERROR: Could not find a version that satisfies the requirement PyPDFLoader (from versions: none)
ERROR: No matching distribution found for PyPDFLoader


In [2]:
%load_ext autoreload
%load_ext watermark
%autoreload 2

In [3]:
import os
from time import time, sleep
from pathlib import Path
import openai
from langchain.callbacks import get_openai_callback
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQAWithSourcesChain, ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

## Initialize the Embedding and setting up enviornment variables

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

## Initialize Language models

In [5]:
from langchain.chat_models import ChatOpenAI

model_name = "gpt-4"

# model_name = "gpt-35-turbo"

llm = ChatOpenAI(engine = model_name, temperature=0)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


## Build RetrieverDB and test queries

In [6]:
def qa(query, llm, retriever, chain_type,verbose=True):
    # create a chain to answer questions 
    qa = RetrievalQA.from_chain_type(
        llm=llm, chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    if verbose:
        print('Q:', result['query'])
        print('A:', result['result'])
    return result

## Test Unstructured PDF

In [11]:
file = "/mnt/NLP/OpenAI/PDF/New_York.pdf"
loader = PyPDFLoader(file)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))

3


In [12]:
top_k = 3
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": top_k})

## Questions and Answers

In [13]:
chain_type = 'stuff'

qa_query = "What are the places to visit Newyork?"

result = qa(qa_query, llm, retriever, chain_type);

Q: What are the places to visit Newyork?
A: There are numerous places to visit in New York City. Some of the most popular attractions include:

1. Statue of Liberty: A symbol of freedom and democracy, this iconic monument is a must-visit.
2. Central Park: A massive urban park in the heart of Manhattan, offering various recreational activities and beautiful scenery.
3. Times Square: Known as "The Crossroads of the World," this bustling area is famous for its bright lights, shopping, and entertainment.
4. Empire State Building: An iconic skyscraper with an observation deck offering stunning views of the city.
5. Broadway: Catch a world-class theater performance at one of the many theaters in this famous district.
6. Metropolitan Museum of Art: One of the largest and most prestigious art museums in the world, featuring a vast collection of art from various periods and cultures.
7. Museum of Modern Art (MoMA): A renowned museum showcasing modern and contemporary art.
8. One World Observato

In [ ]:
qa_query = "List the certifications?"

result = qa(qa_query, llm, retriever, chain_type);